Så prøver vi en gang til at få det her shizzle til at virke. Først alle import

In [2]:
import sys

Sætter Datahandling in i min syspath

In [4]:
sys.path.append('/home/au643300/DataHandling/')

In [5]:
import os
from dask.distributed import Client, as_completed, LocalCluster, get_task_stream
import xarray as xr
from dask_jobqueue import SLURMCluster
import xarray as xr
from dask.distributed import Client, fire_and_forget
import glob
from src.data import *

starter cluster op

In [12]:
cluster=SLURMCluster(cores=16,
                     processes=1,
                     memory="90GB",
                     queue='q64',
                     walltime='0-00:30:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     resources = {'mem':90}
                     )

In [13]:
client=Client(cluster)

In [15]:
cluster.scale(1)

In [14]:
client

Client Scheduler: tcp://10.100.8.11:35574 Dashboard: http://10.100.8.11:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [17]:
raw_path="/home/au643300/DataHandling/data/raw/"


In [27]:
filelist=glob.glob(raw_path+'*.u')
file_path=[]

for path in filelist:
    file_path.append(os.readlink(path))


forkorter lige filelist ned til 30 stk

In [28]:
file_path=file_path[0:30]
len(file_path)

30

In [ ]:
a=client.map(to_netcdf,filelist,retries=3,resources={'mem':5},batch_size=20)

fire_and_forget(a)